In [2]:
import torch
from pytorch_transformers import BertTokenizer, BertForMaskedLM

In [3]:
model            = {}
model['GooBERT'] = BertForMaskedLM.from_pretrained('GooBERT')
model['FinBERT'] = BertForMaskedLM.from_pretrained('FinBERT-Prime_128MSL-250K')
model['PreBERT'] = BertForMaskedLM.from_pretrained('FinBERT-Pre2K_128MSL-250K')
model['ComBERT'] = BertForMaskedLM.from_pretrained('FinBERT-Combo_128MSL-250K')

In [69]:
S1 = '[CLS] the company has a fiduciary duty to its shareholders . [SEP]'
S2 = 'one of its many regulatory requirements . [SEP]'

S1 = '[CLS] the volatility caused us to what [SEP]'
S2 = 'one of its many regulatory requirements . [SEP]'

S1 = '[CLS] we have geographically diversified sales [SEP]'
S2 = 'we believe that we hold the worst market position [SEP]'

S1 = '[CLS] we have geographically diversified sales [SEP]'
S2 = 'we believe that we hold the best market position [SEP]'

S1 = '[CLS] ceo total compensation [SEP]'
S2 = 'resulting from foo of this year [SEP]'

# sentence_1 = ('[CLS] we have geographically [MASK] sales [SEP]', 'diversified')
# sentence_2 = ('we believe that we hold the [MASK] market position [SEP]', 'best')

# sentence_1 = ('[CLS] we have geographically [MASK] sales [SEP]', 'diversified')
# sentence_2 = ('we believe that we hold the [MASK] market position [SEP]', 'best')

# S1 = sentence_1[0]
# S2 = sentence_2[0]

S1 = '[CLS] market conditions have improved since the 2007-2009 recession . [SEP]'
S2 = 'conditions remain challenging for financial institutions . [SEP]'

tokenizer      = BertTokenizer.from_pretrained('bert-base-uncased')
text           = f'{S1} {S2}'
tokenized_text = tokenizer.tokenize(text)

# MI = [i for i, x in enumerate(tokenized_text) if x == "[MASK]"]
# MI = [tokenized_text.index(sentence_1[1]), tokenized_text.index(sentence_2[1])]

for i, word in enumerate(tokenized_text):
    print("{} {}".format(i, word))
    
print(tokenized_text)


0 [CLS]
1 market
2 conditions
3 have
4 improved
5 since
6 the
7 2007
8 -
9 2009
10 recession
11 .
12 [SEP]
13 conditions
14 remain
15 challenging
16 for
17 financial
18 institutions
19 .
20 [SEP]
['[CLS]', 'market', 'conditions', 'have', 'improved', 'since', 'the', '2007', '-', '2009', 'recession', '.', '[SEP]', 'conditions', 'remain', 'challenging', 'for', 'financial', 'institutions', '.', '[SEP]']


In [85]:
import numpy as np

MI = [10, 14]

for i in MI :
    tokenized_text[i] = '[MASK]'

print(tokenized_text)

indexed_tokens   = tokenizer.convert_tokens_to_ids(tokenized_text)
segments_ids     = [0] * len(tokenizer.tokenize(S1)) + [1] * len(tokenizer.tokenize(S2))

print(segments_ids)

tokens_tensor    = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

preds  = {}
for m in model:
    with torch.no_grad():
        preds[m] = model[m](tokens_tensor, token_type_ids = segments_tensors)[0]

        predicted_index = torch.argmax(preds[m][0, 14]).item()

        predicted_indecies = np.argsort((preds[m][0, 10]))
        print(m)
        print("first {}".format(tokenizer.convert_ids_to_tokens([predicted_indecies[-1].item()])))
        print("second {}".format(tokenizer.convert_ids_to_tokens([predicted_indecies[-2].item()])))
        print("third {}".format(tokenizer.convert_ids_to_tokens([predicted_indecies[-3].item()])))
        print("fourth {}".format(tokenizer.convert_ids_to_tokens([predicted_indecies[-4].item()])))
        print("fifth {}".format(tokenizer.convert_ids_to_tokens([predicted_indecies[-5].item()])))
        print("sixth {}".format(tokenizer.convert_ids_to_tokens([predicted_indecies[-6].item()])))
        print("seventh {}".format(tokenizer.convert_ids_to_tokens([predicted_indecies[-7].item()])))
        print("eighth {}".format(tokenizer.convert_ids_to_tokens([predicted_indecies[-8].item()])))
        print("nineth {}".format(tokenizer.convert_ids_to_tokens([predicted_indecies[-9].item()])))
        print("tenth {}".format(tokenizer.convert_ids_to_tokens([predicted_indecies[-10].item()])))
        print("")
        
        
for m in preds:
    tokens = []
    for i in MI:
        predicted_index = torch.argmax(preds[m][0, i]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
        tokens.append(predicted_token)

    print(f'{m} : {tokens}')

['[CLS]', 'market', 'conditions', 'have', 'improved', 'since', 'the', '2007', '-', '2009', '[MASK]', '.', '[SEP]', 'conditions', '[MASK]', 'challenging', 'for', 'financial', 'institutions', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
GooBERT
first ['recession']
second ['period']
third ['season']
fourth ['crisis']
fifth ['year']
sixth ['decade']
seventh ['crash']
eighth ['boom']
nineth ['election']
tenth ['cycle']

FinBERT
first ['recession']
second ['crisis']
third ['period']
fourth ['attacks']
fifth ['hurricanes']
sixth ['levels']
seventh ['peak']
eighth ['announcement']
nineth ['elections']
tenth ['season']

PreBERT
first ['period']
second ['season']
third ['fires']
fourth ['deadline']
fifth ['seasons']
sixth ['blizzard']
seventh ['winter']
eighth ['level']
nineth ['harvest']
tenth ['levels']

ComBERT
first ['recession']
second ['period']
third ['crisis']
fourth ['peak']
fifth ['time']
sixth ['highs']
seventh ['hurricanes']
eighth ['vintage']
nineth 

In [84]:
!ls -a /gold/

.			   FinBERT-Combo_128MSL-500K_512MSL-100K
..			   FinBERT-Pre2K_128MSL-250K
.chunks			   FinBERT-Prime_128MSL-250K
.ipynb_checkpoints	   FinBERT-Prime_128MSL-500K
.kevins			   FinBERT-Prime_128MSL-500K_512MSL-050K
.models			   FinBERT-Prime_128MSL-500K_512MSL-100K
1999.vocab		   FinBERT-Prime_128MSL-500K_512MSL-200K
2019.vocab		   GooBERT
FinBERT-Combo_128MSL-100K  bert.vocab
FinBERT-Combo_128MSL-250K  config.json
FinBERT-Combo_128MSL-500K  tf2pt.sh


Positive examples have better results in Fin, negative Goo performs better.  Financial documents accentuate the positive?

Finbert feels 

